In [45]:
from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import pandas as pd 
from keras.metrics import mean_squared_error

In [52]:
from xgboost import XGBClassifier

/mnt/c/Users/meivenkatkumar/fyp/fyp1/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [46]:
data = pd.read_csv("./data/cleveland_final.csv")

y = data.num.values
x = data.drop(["num"],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42)
print("x_train",len(x_train))
print("x_test",len(x_test))
print("y_train",len(y_train))
print("y_test",len(y_test))



x_train 197
x_test 85
y_train 197
y_test 85


In [48]:
# Set up parameters
time_steps = 20
hidden_units = 49
epochs = 30

# Create a traditional RNN network
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mse', optimizer='adam')
    return model

model_RNN = create_RNN(hidden_units=hidden_units, dense_units=10, input_shape=[1, 49], 
                   activation=['tanh', 'tanh'])
model_RNN.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_13 (SimpleRNN)   (None, 49)                4851      
                                                                 
 dense_8 (Dense)             (None, 10)                500       
                                                                 
Total params: 5,351
Trainable params: 5,351
Non-trainable params: 0
_________________________________________________________________


In [49]:
model_RNN.fit(x_train, y_train, epochs=epochs, batch_size=1, verbose=2)

# Evalute model
train_mse = model_RNN.evaluate(trainX, trainY)
test_mse = model_RNN.evaluate(testX, testY)

# Print error
print("Train set MSE = ", train_mse)
print("Test set MSE = ", test_mse)

Epoch 1/30


ValueError: in user code:

    File "/mnt/c/Users/meivenkatkumar/fyp/fyp1/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/mnt/c/Users/meivenkatkumar/fyp/fyp1/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/mnt/c/Users/meivenkatkumar/fyp/fyp1/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/mnt/c/Users/meivenkatkumar/fyp/fyp1/lib/python3.8/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/mnt/c/Users/meivenkatkumar/fyp/fyp1/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/mnt/c/Users/meivenkatkumar/fyp/fyp1/lib/python3.8/sit

,age,sex,cp,trestbps,htn,chol,cigs,years,fbs,dm,...,lmt,ladprox,laddist,cxmain,om1,rcaprox,rcadist,lvx3,lvx4,lvf
178,43,1,3,130,1,315,30,20,0,-9,...,1,1,1,1,1,1,1,1,1,1
55,54,1,4,124,1,266,0,0,0,-9,...,1,2,1,1,1,1,1,1,1,1
111,56,1,4,125,0,249,5,15,1,1,...,1,2,1,1,1,1,1,1,1,1
217,46,0,4,138,0,243,0,0,0,-9,...,1,1,1,1,1,1,1,1,1,1
38,55,1,4,132,0,353,60,30,0,-9,...,1,2,1,2,1,1,2,1,1,2


In [36]:
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [37]:
def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='mse', optimizer='adam')    
    return model    

model_attention = create_RNN_with_attention(hidden_units=hidden_units, dense_units=1, 
                                  input_shape=x_train.shape[1], activation='tanh')
model_attention.summary()

ValueError: Input 0 of layer "simple_rnn_7" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 49)

In [17]:
model_attention.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)

# Evalute model
train_mse_attn = model_attention.evaluate(trainX, trainY)
test_mse_attn = model_attention.evaluate(testX, testY)

# Print error
print("Train set MSE with attention = ", train_mse_attn)
print("Test set MSE with attention = ", test_mse_attn)

Epoch 1/30
826/826 - 2s - loss: 0.0012 - 2s/epoch - 2ms/step
Epoch 2/30
826/826 - 1s - loss: 0.0011 - 1s/epoch - 2ms/step
Epoch 3/30
826/826 - 1s - loss: 0.0011 - 1s/epoch - 2ms/step
Epoch 4/30
826/826 - 1s - loss: 0.0011 - 1s/epoch - 2ms/step
Epoch 5/30
826/826 - 1s - loss: 0.0010 - 1s/epoch - 2ms/step
Epoch 6/30
826/826 - 1s - loss: 9.6462e-04 - 1s/epoch - 2ms/step
Epoch 7/30
826/826 - 1s - loss: 9.0222e-04 - 1s/epoch - 2ms/step
Epoch 8/30
826/826 - 1s - loss: 8.4075e-04 - 1s/epoch - 2ms/step
Epoch 9/30
826/826 - 1s - loss: 7.6342e-04 - 1s/epoch - 2ms/step
Epoch 10/30
826/826 - 1s - loss: 6.9055e-04 - 1s/epoch - 2ms/step
Epoch 11/30
826/826 - 1s - loss: 6.2542e-04 - 1s/epoch - 2ms/step
Epoch 12/30
826/826 - 1s - loss: 5.6051e-04 - 1s/epoch - 2ms/step
Epoch 13/30
826/826 - 1s - loss: 4.8784e-04 - 1s/epoch - 2ms/step
Epoch 14/30
826/826 - 1s - loss: 4.1636e-04 - 1s/epoch - 2ms/step
Epoch 15/30
826/826 - 1s - loss: 3.5915e-04 - 1s/epoch - 2ms/step
Epoch 16/30
826/826 - 1s - loss: 2.9920

In [53]:
model = XGBClassifier()
model.fit(x_train, y_train)

/mnt/c/Users/meivenkatkumar/fyp/fyp1/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/mnt/c/Users/meivenkatkumar/fyp/fyp1/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[17:53:55] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [54]:
preds = model.predict(x_test)

In [55]:
from sklearn.metrics import classification_report

In [58]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        47
           1       0.54      0.54      0.54        13
           2       0.50      0.57      0.53         7
           3       0.89      0.62      0.73        13
           4       1.00      1.00      1.00         5

    accuracy                           0.84        85
   macro avg       0.77      0.75      0.75        85
weighted avg       0.84      0.84      0.83        85

